<a href="https://colab.research.google.com/github/conuko/AI-Guild/blob/main/Session_3_Optimizing_Toast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Explanation of the different solutions

###EASY:

This implementation loops through all possible values of toast_duration
and wait_duration from 1 to 100, calculates the utility for each combination,
and keeps track of the parameters with the highest utility. It returns the
best parameters once all combinations have been tested.

Note that this approach may take some time to run since it needs to test
10,000 combinations. However, given the simplicity of the problem and the fact
that the function evaluations are fast, this approach should be sufficient.

###MEDIUM (Hill climbing algorithm):
This implementation starts with a random point in the search space, and repeatedly generates neighboring points and moves to the one with the highest utility, as long as it is better than the current point. The algorithm terminates when it reaches a local maximum, i.e., when all neighboring points have lower utility than the current point.

Note that this implementation only searches for local optima, and may not
find the global optimum if it is far from the starting point. To mitigate this, one could run the algorithm multiple times with different starting points and take the best result.

###HARD (Gradient ascent algorithm):
The `find_maximum` function is an implementation of the gradient ascent algorithm to find the optimal values of the `toast_duration`, `wait_duration`, and `power` parameters that maximize the `utility` function.

The function takes three optional parameters: `learning_rate`, `max_iterations`, and `tolerance`. `learning_rate` controls the step size of the algorithm, `max_iterations` limits the maximum number of iterations, and tolerance sets the convergence threshold.

The algorithm initializes the parameters toast_duration, wait_duration, and power to some initial values. It then calculates the initial `utility` value using the utility function.

Next, it enters a loop that repeats until one of two stopping conditions is met. The first condition is that the change in utility value is smaller than tolerance, which indicates that the algorithm has converged to a local maximum. The second condition is that the number of iterations has exceeded `max_iterations`.

In each iteration of the loop, the algorithm calculates the gradient of the utility function with respect to each parameter using the partial derivative of the utility function with respect to that parameter. It then updates each parameter by adding the gradient times the learning rate. This update moves the parameters in the direction of the gradient and increases the utility value.

After updating the parameters, the algorithm calculates the new utility value and the change in utility value. If the change in utility value is less than the tolerance, the algorithm stops. Otherwise, it updates the current utility value and continues to the next iteration.

Finally, the algorithm returns the values of toast_duration, wait_duration, and power that maximize the utility function. These values are the ones that were reached in the last iteration of the loop.

It is worth noting that the gradient ascent algorithm is a local search algorithm, and it may converge to a local maximum rather than a global maximum. In this case, the initial values of the parameters could be randomly initialized and the algorithm run multiple times to increase the chances of finding a global maximum.



In [17]:
################################################################################
# Problem Setup: You want the perfect bread
# What influences this?
#  - how long do you toast it?
#  - how long after toasting do you eat the bread?
#  - Do you have power? And how much? 
#  - Which toaster do you use?
################################################################################

import math
import random

In [7]:
################################################################################
# the function you are supposed to optimize.
# It has the following input:
#  toast_duration: duration of toasting in seconds. It is supposed to be an integer between 1 and 100
#  wait_duration: duration of waiting after toasting in seconds. It's supposed to be an integer between 1 and 100
#  toaster: the number of the toaster you want to use. It's supposed to be an integer, between 1 and 10.
#  power: how much power the toaster has (it's supposed to be a floating point number between 0 and 2)
################################################################################
def utility(toast_duration, wait_duration, power = 1.0,toaster = 1):
    # handle input errors
    if (not type(toast_duration) is int) and not (1 <= toast_duration <= 100):
        raise ValueError("toast_duration is not an integer")
    if (not type(wait_duration) is int) and not (1 <= wait_duration <= 100):
        raise ValueError("wait_duration is not an integer")
    if (not type(toaster) is int) and not (1 <= toaster <= 10):
        raise ValueError("toaster is not an integer or is not in a valid range")
    if (not type(power) is float) and not (0.0 <= power <= 2.0):
        raise ValueError("power is not a float or not in the valid range")

    # get toaster specific configuration
    hpt = [10,8,15,7,9,2,9,19,92,32][toaster-1]
    hpw = [1,4,19,3,20,3,1,4,1,62][toaster-1]
    toaster_utility = [1,0.9,0.7,1.3,0.3,0.8,0.5,0.8,3,0.2][toaster-1]

    # calculate values
    toast_utility = -0.1*(toast_duration-hpt)**2+1
    wait_utility = -0.01*(wait_duration-hpw)**2+1
    overall_utility = (toast_utility + wait_utility) * toaster_utility

    # apply modifier based on electricity
    power_factor = math.sin(10*power+math.pi/2 -10) + power*0.2
    overall_utility *= power_factor

    return overall_utility

In [59]:
################################################################################
# Writing this function is your homework. 
# The function should return the tuple of parameters that optmizes the function.
# 
# You can implement it in multiple difficulty levels:
# easy: 
#     - implement it with only two parameters: toast_duration and wait_duration
#     - e.g., utility(2,3)
#     - Implement the function by testing all possible values for these variables.
#     - (This state space has only 10000 values, so it shouldn't take too long)
#
# medium: 
#    - same as easy, but implement hill climbing
#    - see pseudo code
# hard: 
#    - also use the parameter power
#    - e.g., utility(2,3,1.2)
#    - this introduces the following complications:
#        - multiple maxima
#        - a continuous parameter
#    - implement gradient ascent
# very hard:
#    - Same as hard, but use repeated search to find all maxima. 
#    - repeated search: 
#        - apply gradient descent from different starting points.
#    - I think there are 5 maxima. But I'm not sure :-P
# prepare to cry:
#    - find the optimum for all four parameters
#    - define your own algorithm!

################################################################################
# EASY:
def find_maximum():
    # initialize maximum utility and corresponding parameters
    max_utility = -float("inf")
    max_params = None
    
    # loop over all possible parameter values
    for toast_duration in range(1, 101):
        for wait_duration in range(1, 101):
            # calculate utility for these parameters
            params = (toast_duration, wait_duration)
            curr_utility = utility(*params)
            
            # update maximum if necessary
            if curr_utility > max_utility:
                max_utility = curr_utility
                max_params = params
                
    return max_params

################################################################################
# MEDIUM (HILL CLIMBING):
def find_maximum_medium():
    # Start with a random point in the search space
    curr_params = (random.randint(1, 100), random.randint(1, 100))
    curr_utility = utility(*curr_params)

    # Define a function to generate neighboring points
    def get_neighbors(params):
        neighbors = []
        for delta_t in [-1, 1]:
            for delta_w in [-1, 1]:
                t = params[0] + delta_t
                w = params[1] + delta_w
                if 1 <= t <= 100 and 1 <= w <= 100:
                    neighbors.append((t, w))
        return neighbors

    # Define a loop to perform hill climbing
    while True:
        # Generate neighbors and find the one with the highest utility
        neighbors = get_neighbors(curr_params)
        best_neighbor = max(neighbors, key=lambda x: utility(*x))

        # If the best neighbor has higher utility, move to that point
        if utility(*best_neighbor) > curr_utility:
            curr_params = best_neighbor
            curr_utility = utility(*curr_params)
        # Otherwise, return the current point as the optimum
        else:
            return curr_params

################################################################################
# HARD (GRADIENT ASCENT):
def find_maximum_hard(learning_rate=0.01, max_iterations=1000, tolerance=1e-6):
    # set initial values for the parameters
    toast_duration = 50
    wait_duration = 50
    power = 1.0

    # set the initial utility and the delta utility
    current_utility = utility(toast_duration, wait_duration, power)
    delta_utility = tolerance + 1

    # iterate until convergence or max_iterations
    iteration = 0
    while delta_utility > tolerance and iteration < max_iterations:
        # calculate the gradient for each parameter
        d_toast_duration = 0.2*(toast_duration-10)
        d_wait_duration = 0.02*(wait_duration-19)
        d_power = math.cos(10*power+math.pi/2 -10) + 0.2

        # update the parameters using the gradient
        toast_duration += learning_rate * d_toast_duration
        wait_duration += learning_rate * d_wait_duration
        power += learning_rate * d_power

        # calculate the new utility and delta utility
        new_utility = utility(toast_duration, wait_duration, power)
        delta_utility = new_utility - current_utility
        current_utility = new_utility

        iteration += 1

    return (toast_duration, wait_duration, power)

################################################################################
# HARD SECOND APPROACH (GRADIENT ASCENT):
# To handle multiple maxima, we can add random initialization to the parameters
# in the find_maximum function. This way, we can run the gradient ascent
# algorithm multiple times from different starting points and choose the
# parameter values that yield the highest utility value. Additionally,
# to handle a continuous parameter, we can modify the function to take
# a range of values for that parameter, and iterate over that range to find the
# optimal value. Here's an improved version of the find_maximum_hard function:

def find_maximum_hard2(learning_rate=0.01, max_iterations=1000, tolerance=1e-6):
    # set the range of values for the continuous parameter
    power_range = [0.1 * i for i in range(21)]

    # set initial values for the parameters
    toast_duration = 50
    wait_duration = 50
    power = 1.0

    # set the initial utility and the delta utility
    current_utility = utility(toast_duration, wait_duration, power)
    delta_utility = tolerance + 1

    # iterate until convergence or max_iterations
    iteration = 0
    while delta_utility > tolerance and iteration < max_iterations:
        # calculate the gradient for each parameter
        d_toast_duration = 0.2*(toast_duration-10)
        d_wait_duration = 0.02*(wait_duration-19)
        d_power = math.cos(10*power+math.pi/2 -10) + 0.2

        # update the parameters using the gradient
        toast_duration += learning_rate * d_toast_duration
        wait_duration += learning_rate * d_wait_duration
        power += learning_rate * d_power

        # make sure the parameter values are within the valid range
        toast_duration = max(min(toast_duration, 100), 1)
        wait_duration = max(min(wait_duration, 100), 1)
        power = max(min(power, 2.0), 0.0)

        # calculate the new utility and delta utility
        new_utility = utility(toast_duration, wait_duration, power)
        delta_utility = new_utility - current_utility
        current_utility = new_utility

        iteration += 1

    # initialize the maximum utility and parameter values
    max_utility = current_utility
    max_toast_duration = toast_duration
    max_wait_duration = wait_duration
    max_power = power

    # iterate over multiple random initializations to handle multiple maxima
    for i in range(10):
        # choose a random power value
        power = random.choice(power_range)

        # set initial values for the other parameters
        toast_duration = random.randint(1, 100)
        wait_duration = random.randint(1, 100)

        # set the initial utility and the delta utility
        current_utility = utility(toast_duration, wait_duration, power)
        delta_utility = tolerance + 1

        # iterate until convergence or max_iterations
        iteration = 0
        while delta_utility > tolerance and iteration < max_iterations:
            # calculate the gradient for each parameter
            d_toast_duration = 0.2*(toast_duration-10)
            d_wait_duration = 0.02*(wait_duration-19)
            d_power = math.cos(10*power+math.pi/2 -10) + 0.2

            # update the parameters using the gradient
            toast_duration += learning_rate * d_toast_duration
            wait_duration += learning_rate * d_wait_duration
            power += learning_rate * d_power

            # make sure the parameter values are within the valid range
            toast_duration = max(min(toast_duration, 100), 1)
            wait_duration = max(min(wait_duration, 100), 1)
            power = max(min(power, 2.0), 0.0)

            # calculate the new utility and delta utility
            new_utility = utility(toast_duration, wait_duration, power)
            delta_utility = new_utility - current_utility
            current_utility = new_utility

            iteration += 1

            # update the maximum utility and parameter values if necessary
            if current_utility > max_utility:
                max_utility = current_utility
                max_toast_duration = toast_duration
                max_wait_duration = wait_duration
                max_power = power

        return (max_toast_duration, max_wait_duration, max_power)



In [53]:
# use the function and see what it thinks the optimum is
optimum = find_maximum()
print("Optimum:",optimum,)
print("value:",utility(*optimum))

Optimum: (10, 1)
value: 2.4


In [54]:
# use the function and see what it thinks the optimum is
optimum = find_maximum_medium()
print("Optimum:",optimum,)
print("value:",utility(*optimum))

Optimum: (10, 6)
value: 2.1


In [63]:
# use the function and see what it thinks the optimum is
optimum = find_maximum_hard()
print("Optimum:",optimum,)
print("value:",utility(*optimum))

Optimum: (50.08, 50.0062, 1.002)
value: -219.22459222225206


In [68]:
# use the function and see what it thinks the optimum is
optimum = find_maximum_hard2()
print("Optimum:",optimum,)
print("value:",utility(*optimum))

Optimum: (30.04, 5.9974, 0.10612118485241757)
value: 33.14731439244762
